In [ ]:
import pandas as pd
import psycopg2 

In [ ]:
labels = pd.read_csv("data/labels.csv")
# pull heart-failure flags from CDM
conn = psycopg2.connect(
    host="pg-2vge6o.vpc-pub-cdb-kr.gov-ntruss.com",
    port=5432,
    database="cdm",
    user="mindy_lee",
    password="Nstri05!@"
)

hf_sql = """
SELECT
  person_id,
  CASE WHEN COUNT(*) > 0 THEN 1 ELSE 0 END AS hf_outcome
FROM cdm_public.condition_occurrence_v
WHERE condition_concept_id IN (
    316139,  -- heart failure
    316140,  -- acute heart failure
    315831   -- congestive heart failure
)
GROUP BY person_id
"""
hf_df = pd.read_sql(hf_sql, conn)
conn.close()
labels = labels.merge(hf_df, on="person_id", how="left")
labels["hf_outcome"] = labels["hf_outcome"].fillna(0).astype(int)
labels.to_csv("data/labels_with_hf.csv", index=False)
print(labels.head())
print("Counts of outcomes:\n", labels["hf_outcome"].value_counts())